# 1. Install LangChain Packages

In [ ]:
# Install necessary packages
!pip install langchain chromadb sentence-transformers
!pip install langchain-community
!pip install langchain-nvidia-ai-endpoints

# 2. Use NVIDIA NIM API
- [Get NVIDIA NIM API here](https://build.nvidia.com/explore/discover)
- [General RAG tutorial here](https://github.com/Squirtle007/Retrieval-Augmented_Generation)

In [2]:
import os

os.environ['NVIDIA_API_KEY'] = "Paste-Your-API-Here"

# 3. Import Modules

In [3]:
# Import necessary modules
import sqlite3
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# If you are using Google Colab, use this to upload the `cpr.db` database.
#from google.colab import files
#files.upload()

# 4. Prepare Data and Generate Vector Embeddings

- Ensure the `cpr.db` database file is in the current working directory.
- Connect to the SQLite database and retrieve problem-solution pairs from the `problem_solution_pairs` table.
- Format the retrieved data into a list of dictionaries, each containing an ID and combined problem-solution text.
- Use the `intfloat/e5-large-v2` model from Hugging Face to generate embeddings for the text data.
- Initialize a Chroma vector store with the generated embeddings and associate metadata (problem IDs) for retrieval tasks.

In [6]:
# Connect to the SQLite database
DB_PATH = "./cpr.db"
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Retrieve problem-solution pairs from the database
def get_problems_from_db():
    cursor.execute("SELECT id, problems, solutions FROM problem_solution_pairs")
    return cursor.fetchall()

problems = get_problems_from_db()

# Prepare problem-solution data
problem_solution_pairs = [
    {"id": doc[0], "text": f"Problem:\n{doc[1]}\n\nSolution:\n{doc[2]}"}
    for doc in problems
]

In [7]:
# Generate embeddings and initialize vectorstore
embedding_model = "intfloat/e5-large-v2"
hf_embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
texts = [ps["text"] for ps in problem_solution_pairs]
vectorstore = Chroma.from_texts(
    texts=texts,
    embedding=hf_embeddings,
    metadatas=[{"id": ps["id"]} for ps in problem_solution_pairs]
)

<ipython-input-7-6c0c434b5606>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab 

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

# 5. Define a Retrieval-Enhanced Prompt Template

- Create a custom `ChatPromptTemplate` tailored for competitive programming tasks.
- Set the system's role to focus on algorithm optimization and context analysis.
- Feel free to play around with the instructions.

In [15]:
# Define Retrieval-Enhanced Prompt Template
retrieval_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an AI specialized in competitive programming and algorithm optimization."),
    ("system", "Use your reasoning skills to analyze the provided context, discerning relevant and irrelevant information."),
    ("system", "Select one or more relevant code snippets. If no snippet directly applies, adapt the logic of the context to make it relevant to the query."),
    ("system", "Provide a concise explanation of the selected code's main idea and explicitly include the code snippet(s)."),
    ("system", "Using your domain knowledge, specify the time complexity of the code snippet(s)."),
    ("system", "Begin your response with the emoticon: 🤓☝️."),
    ("user", "{input}")
])

# 6. Integrate an LLM and Build a Retrieval-Enhanced Answer Chain

- **Define the LLM:** Use the `meta/llama3-8b-instruct` foundation model with fine-tuned parameters:
  - Set a low temperature (`0.1`) for deterministic responses.
  - Limit the output to 500 tokens for concise answers.
- **Create a processing chain:** Use the pipe operator (`|`) to connect:
  - The retrieval-enhanced prompt template.
  - The LLM.
  - A `StrOutputParser` to format the output.
- **Retrieve top-ranked results:** Implement a function to:
  - Embed the query using the embedding model.
  - Search the vector store for the top `k` most similar problem-solution pairs.
- **Generate an answer with ranking:**
  - Compile a ranked context from the retrieved problem-solution pairs.
  - Prepare input for the processing chain by combining the query and context.
  - Invoke the chain to generate a response based on the ranked context.
- **Note:** This setup is a suggestion; feel free to adjust the parameters to fit your needs.

In [16]:
# Define the LLM (using a custom prompt)
foundation_model = "meta/llama3-8b-instruct"  # Use the recommended model
llm = ChatNVIDIA(model=foundation_model, temperature=0.1, max_tokens=500)

# Define the chain using the pipe operator
chain = retrieval_prompt | llm | StrOutputParser()
print(type(chain))

# Retrieve top-ranked problem-solution pairs
def retrieve_top_ranked(query, vectorstore, top_k=5):
    query_embedding = hf_embeddings.embed_query(query)
    results = vectorstore.similarity_search_with_score(query, k=top_k)
    return results

# Generate an answer using the retrieval-enhanced chain
def generate_answer_with_ranking(query, vectorstore, chain, top_k=5):
    retrieval_results = retrieve_top_ranked(query, vectorstore, top_k)
    ranked_context = "\n\n".join(
        [f"Problem-Solution Pair:\n{result[0].page_content}" for result in retrieval_results]
    )
    # Prepare input for the chain
    chain_input = {
        "input": f"Query: {query}\n\nContext:\n{ranked_context}\n\nAnswer:"
    }
    # Invoke the chain to process the input
    response = chain.invoke(chain_input)
    return response, retrieval_results

<class 'langchain_core.runnables.base.RunnableSequence'>


# 7. Test the Pipeline

In [17]:
# Test the pipeline
# Enter 'quit' or 'exit' in query to exit the loop
while True:
  query = input("Query: ")
  if query == "quit" or query == "exit":
    break
  response, ranked_context = generate_answer_with_ranking(query, vectorstore, chain)
  print("Answer:", response)
  print("\n\n")
  #print("\n-----\nFor Debugging:\n-----\n")
  #for doc in ranked_context:
  #    print(doc[0].metadata, "\n", doc[0].page_content, "\n---")

Query: How to find the sum of all the minimal product-sum  for 2 <= k <= 15000?
Answer: 🤓☝️

To find the sum of all the minimal product-sum numbers for 2 <= k <= 15000, we can use the following Python code:

```python
LIMIT = 15000
min_sum_product = [None] * (LIMIT + 1)

def factorize(n, remain, max_factor, sum_factors, num_terms):
    if remain == 1:
        if sum_factors > n:
            pass
        num_terms += n - sum_factors
        if num_terms <= LIMIT and (min_sum_product[num_terms] is None or n < min_sum_product[num_terms]):
            min_sum_product[num_terms] = n
    else:
        for i in range(2, max_factor + 1):
            if remain % i == 0:
                factor = i
                factorize(n, remain // factor, min(factor, max_factor), sum_factors + factor, num_terms + 1)

for i in range(2, LIMIT * 2 + 1):
    factorize(i, i, i, 0, 0)

result = sum(set(min_sum_product[2:]))
print(result)
```

This code uses a recursive function `factorize` to find all minimal pro